The present dashboarding hands on lab uses [Airline Reporting Carrier On-Time Performance dataset](https://developer.ibm.com/exchanges/data/all/airline/) from [Data Asset eXchange](https://developer.ibm.com/exchanges/data/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-wwwcourseraorg-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDV0101ENSkillsNetwork20297740-2021-01-01&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork-20297740&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ).   

Glossary of the dataset could be found from the link below: [Link](https://dax-cdn.cdn.appdomain.cloud/dax-airline/1.0.1/data-preview/index.html?_ga=2.83976473.1025912384.1640558225-1040743284.1639660199)

This hands on practice is a part of [Data Visualization with Python](https://www.coursera.org/learn/python-for-data-visualization?) course offered by [Coursera.org](https://www.coursera.org/).

In [1]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import matplotlib as mtp
import pandas as pd
import seaborn as sns
import numpy as np
import warnings
import seaborn as sns
import matplotlib.patches as mpatches
from wordcloud import WordCloud, STOPWORDS
import urllib
from PIL import Image
import folium
from folium import plugins
import wget
import plotly.graph_objects as go
import plotly.express as px
import plotly
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash import no_update
from dash.dependencies import Input,Output
import gunicorn
%matplotlib inline

In [2]:
# importing the csv dataset as a dataframe using pandas library
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/airline_data.csv'
airline_data =  pd.read_csv(url,encoding = "ISO-8859-1")

In [5]:
app = dash.Dash(__name__) # creating a dash app
server = app.server # this is used for deploying the dashboard on a web using Heroku
app.layout = html.Div([
    # creating the heading
    html.H1('US Domestic Airline Flights Performance',style={'textAlign':'center','color':'#503D36','font-size':40}),
    # creating the drop downs 
    html.Div([html.Label(['Report Type:'], style={'font-size':30}),
        dcc.Dropdown(id='inp01',options=[{'label':'Yearly Airline Performance Report','value':'OPT1'},
                                         {'label':'Yearly Airline Delay Report','value':'OPT2'}],
                     style={'width':'80%','padding':'3px','font-size':'20px','text-align-last':'center'}),
                  html.Label(['Choose year:'], style={'font-size':30}),
              dcc.Dropdown(id='inp02',
                           options=[{'label':'2005','value':'2005'},
                                    {'label':'2006','value':'2006'},
                                    {'label':'2007','value':'2007'},
                                    {'label':'2008','value':'2008'},
                                    {'label':'2009','value':'2009'},
                                    {'label':'2010','value':'2010'},
                                    {'label':'2011','value':'2011'},
                                    {'label':'2012','value':'2012'},
                                    {'label':'2013','value':'2013'},
                                    {'label':'2014','value':'2014'},
                                    {'label':'2015','value':'2015'},
                                    {'label':'2016','value':'2016'},
                                    {'label':'2017','value':'2017'},
                                    {'label':'2018','value':'2018'},
                                    {'label':'2019','value':'2019'},
                                    {'label':'2020','value':'2020'},],
                           style={'width':'80%','padding':'3px','font-size':'20px','text-align-last':'center'},),]),
    # creating 5 graphs in 3 rows
    html.Div([dcc.Graph(id='plot01')],style={'border':'solid'}),
    html.Div([dcc.Graph(id='plot02'),dcc.Graph(id='plot03')],style={'display':'flex','border':'solid'}),
    html.Div([dcc.Graph(id='plot04'),dcc.Graph(id='plot05')],style={'display':'flex','border':'solid'}),
])
#########################################################################
# creating a decorator to add app application to the Graph function defined below
# the ids for inputs and outputs are the same as those provided in the app layout
@app.callback([Output(component_id='plot01',component_property='figure'),
               Output(component_id='plot02',component_property='figure'),
               Output(component_id='plot03',component_property='figure'),
               Output(component_id='plot04',component_property='figure'),
               Output(component_id='plot05',component_property='figure')],
              [Input(component_id='inp01',component_property='value'),
              Input(component_id='inp02',component_property='value')])
#########################################################################
def Graph(inp01,inp02):
    filter_yr = airline_data[airline_data['Year']==int(inp02)]
    if inp01=='OPT1':
        
        # creating a tree map
        airline_gb1 = filter_yr.groupby(['Reporting_Airline','DestState'],as_index=False)['Flights'].sum()
        fig01 = px.treemap(airline_gb1,path=['Reporting_Airline','DestState'],color ='Reporting_Airline', values='Flights',)
        fig01.update_layout(title_text='Number of flights flying to each state from each reporting airline',title_x = 0.5)
        
        # creating a pie plot
        airline_gb2 = filter_yr.groupby(['Reporting_Airline'],as_index=False)['Diverted'].sum()
        airline_gb2_filter = airline_gb2[airline_gb2['Diverted']!=0]
        fig02 = px.pie(airline_gb2_filter, values='Diverted',names='Reporting_Airline')
        fig02.update_layout(title_text='Percentage of diverted airport landings per reporting airline',title_x = 0.5)

        # creating a Choropleth map
        airline_gb3 = filter_yr.groupby('OriginState')['Flights'].sum().reset_index()
        fig03 = go.Figure(data=go.Choropleth(
            locations=airline_gb3['OriginState'], # Spatial coordinates
            z = airline_gb3['Flights'].astype(float), # Data to be color-coded
            locationmode = 'USA-states', # set of locations match entries in `locations`
            colorscale = 'YlGn',
            colorbar_title = "Flight number",))
        fig03.update_layout(title_text = 'Number of flights flying from each state',geo_scope='usa') 

        # creating a bar plot
        airline_gb4 = filter_yr.groupby(['Month','CancellationCode'])['Flights'].sum().reset_index()
        fig04 = px.bar(airline_gb4,x = 'Month', y = 'Flights',color = 'CancellationCode',
                       title='Number of flights under different cancellation categories')

        # creating a line plot
        airline_gb5 = filter_yr.groupby(['Month','Reporting_Airline'])['AirTime'].mean().reset_index()
        fig05 = px.line(airline_gb5,x='Month',y='AirTime',color ='Reporting_Airline',title='Average monthly flight time (minutes) by airline')
        return [fig01,fig02,fig03,fig04,fig05]

    elif inp01=='OPT2':
        # creating 5 line plots
        # plot No. 01
        airline_gb1 = filter_yr.groupby(['Month','Reporting_Airline'])['CarrierDelay'].mean().reset_index()
        fig01 = px.line(airline_gb1,x='Month',y='CarrierDelay',color ='Reporting_Airline',title ='Monthly average carrier delay (minutes)')
        fig01.update_yaxes(title_text='Carrier delay - minutes')
        # plot No. 02
        airline_gb2 = filter_yr.groupby(['Month','Reporting_Airline'])['WeatherDelay'].mean().reset_index()
        fig02 = px.line(airline_gb2,x='Month',y='WeatherDelay',color ='Reporting_Airline',title ='Monthly average weather delay (minutes)')
        fig02.update_yaxes(title_text='Weather delay - minutes')
        # plot No. 03
        airline_gb3 = filter_yr.groupby(['Month','Reporting_Airline'])['NASDelay'].mean().reset_index()
        fig03 = px.line(airline_gb3,x='Month',y='NASDelay',color ='Reporting_Airline',title ='Monthly average national air system delay (minutes)')
        fig03.update_yaxes(title_text='National air system delay - minutes')
        # plot No. 04
        airline_gb4 = filter_yr.groupby(['Month','Reporting_Airline'])['SecurityDelay'].mean().reset_index()
        fig04 = px.line(airline_gb4,x='Month',y='SecurityDelay',color ='Reporting_Airline',title ='Monthly average security delay (minutes)')        
        fig04.update_yaxes(title_text='Security delay - minutes')
        # plot No. 05
        airline_gb5 = filter_yr.groupby(['Month','Reporting_Airline'])['LateAircraftDelay'].mean().reset_index()
        fig05 = px.line(airline_gb5,x='Month',y='LateAircraftDelay',color ='Reporting_Airline',title ='Monthly average late aircraft delay (minutes)')
        fig05.update_yaxes(title_text='Late aircraft delay - minutes')

        return [fig01,fig02,fig03,fig04,fig05]
#########################################################################
app.run_server(port=8002,host='127.0.0.1',debug=False)

Dash is running on http://127.0.0.1:8002/

Dash is running on http://127.0.0.1:8002/

Dash is running on http://127.0.0.1:8002/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8002/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Dec/2021 20:48:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2021 20:48:58] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2021 20:48:58] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2021 20:48:58] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2021 20:48:58] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2021 20:48:58] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\farha\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\farha\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\farha\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\farha\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\farha\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\farha\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\farha\anaconda3\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_d

127.0.0.1 - - [27/Dec/2021 20:48:58] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\farha\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\farha\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\farha\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\farha\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\farha\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\farha\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\farha\anaconda3\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_d

127.0.0.1 - - [27/Dec/2021 20:49:00] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [27/Dec/2021 20:49:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2021 20:49:08] "POST /_dash-update-component HTTP/1.1" 200 -
